In [ ]:
 import requests
from lxml import html
import smtplib, ssl
import time
from bs4 import BeautifulSoup
import pandas as pd
import csv
from datetime import datetime
from google.colab import files
import pytz



In [ ]:
def check_price(URL):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
        'Accept-Encoding':  None
    }
    
    page = requests.get(URL,headers=headers)
    tree = html.fromstring(page.content)
    
    # open devolper option for amazon and note the class name or id name for the price
    
    price = tree.xpath('//span[@class="a-price-whole"]/text()')[0]
    
    return int(float(price.replace(',','')))


In [ ]:
def email_price(url,price,site):
    
    # please read this. As less securtiy option is not working in google. you have to on two step verification and next go to App password and set a password
    
    # add the mail and password
    smtp_server = "smtp.gmail.com"
    port = 587  # For starttls
    
    #Enter your mail id
    sender_email = "rahulsandireddy03@gmail.com"
    
    # Enter your password
    password = \n
    # Create a secure SSL context
    context = ssl.create_default_context()
    if(site == "amazon"):
      pm=pro_name(url)
    else:
      pm=flip_name(url)
    # Try to log in to server and send email
    try:
        server = smtplib.SMTP(smtp_server, port)
        server.ehlo()  # Can be omitted
        server.starttls(context=context)  # Secure the connection
        server.ehlo()  # Can be omitted
        server.login(sender_email, password)
        subject = "Price fell down!"
        body=url
        message= f"Subject: {subject}\n\n{pm}\n\n{body}\n\n present price is {price}"
        server.sendmail(sender_email,sender_email,message)

    except Exception as e:
        # Print any error messages to stdout
        print(e)
    finally:
        server.quit()



In [ ]:
def pro_name(url):
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
      'Accept-Encoding': None
  }     
  
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, "html.parser")
  product_name = soup.find("span", {"id": "productTitle"}).text.strip()
  return product_name



In [ ]:
def flip_name(url):
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
      'Accept-Encoding': None
  }     
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, "html.parser")
  product_name = soup.find("span", {"class": "B_NuCI"}).text.strip()
  return product_name

In [ ]:

def write_to_csv(product_name, price,st):
    filename = "/content/drive/MyDrive/web_mining/Cost/{}.csv".format(product_name)
    headers = ['Date', 'Time', 'Price','Site']

    india_tz = pytz.timezone('Asia/Kolkata')
    current_time = datetime.now(india_tz)

    rows = [[datetime.now().strftime("%Y-%m-%d"), current_time.strftime("%H:%M:%S"), price,st]]
    # if the file does not exist, create a new file with headers
    try:
        with open(filename, 'x') as f:
            writer = csv.writer(f)
            writer.writerow(headers)
            writer.writerows(rows)
    except FileExistsError:
        # if the file already exists, append new data to it
        with open(filename, 'a') as f:
            writer = csv.writer(f)
            writer.writerows(rows)



In [ ]:
def collect_data(url,pm):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36',
        'Accept-Encoding': None
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    review_section = soup.find("div", {"id": "reviews-medley-footer"})
    review_links = review_section.find_all("a")

    for link in review_links:
        href = link.get("href")
        if href.startswith("http" + "&pageNumber="):
            url1 = href + "&pageNumber="
        else:
            url1 = "https://www.amazon.in" + href + "&pageNumber="
    # creating the dataframe
    cust_name = []
    rev_date = []
    ratings = []
    rev_title = []
    rev_content = []
    for page in range(1, 50):
        new_url = url1 + str(page)
        code = requests.get(new_url, headers=headers)
        if str(code) == "<Response [200]>":
            soup = BeautifulSoup(code.content, 'html.parser')
            names = soup.select('span.a-profile-name')[2:]
            titles = soup.select('a.review-title span')
            dates = soup.select('span.review-date')[2:]
            stars = soup.select('i.review-rating span.a-icon-alt')[2:]
            reviews = soup.select('span.review-text-content span')
            h = min(len(titles), len(dates), len(stars), len(reviews))
            for i in range(h):
                cust_name.append(names[i].get_text())
                rev_date.append(dates[i].get_text().replace("Reviewed in India on ", ""))
                ratings.append(stars[i].get_text().replace("out of 5 stars",""))
                rev_title.append(titles[i].get_text())
                rev_content.append(reviews[i].get_text().strip("\n "))
    df = pd.DataFrame()
    df['Customer Name'] = cust_name
    df['Date'] = rev_date
    df['Ratings'] = ratings
    df['Review Title'] = rev_title
    df['Reviews'] = rev_content
    pm = "Reviews_"+pm
    filename = "/content/drive/MyDrive/web_mining/Reviews/{}.csv".format(pm)
    df.to_csv(filename)
    from google.colab import files
    files.download(filename)
    


In [ ]:
def repl(pm):
    pm=pm.replace(" ","_")
    pm=pm.replace("-","_")
    pm=pm.replace("/","_")
    pm=pm.replace("\\","_")
    return pm

In [ ]:
def flip_check_price(url):
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
      'Accept-Encoding':  None
  }
  page = requests.get(url,headers=headers)
  tree = html.fromstring(page.content)
    
  # open devolper option for amazon and note the class name or id name for the price
    
  price = tree.xpath('//div[@class="_30jeq3 _16Jk6d"]/text()')[0]
  value_str = price.replace('₹', '').replace(',', '') # remove the rupee symbol
  value_int = int(value_str) # convert to integer
  return value_int 
   

In [ ]:
import os
amaz_url = "https://www.amazon.in/Logitech-B170-Wireless-Mouse-Black/dp/B01J0XWYKQ/ref=sr_1_15?brr=1&qid=1680848382&rd=1&sr=8-15"
flip_url= "https://www.flipkart.com/lenovo-130-wireless-optical-mouse/p/itm4a34f5ec3163d?pid=ACCG5DKG6MJEZJCF&lid=LSTACCG5DKG6MJEZJCF1QSM0V&marketplace=FLIPKART&q=Logitech+B170+Wireless+Mouse&store=6bo%2Fai3%2F2ay&srno=s_1_1&otracker=search&otracker1=search&fm=Search&iid=en_5nypMKT1kBwGFRmFAzFWYP1xAFp8pD7b1oRir%2BEbyKcG1zBAcqorbbRhcYsPo2rVxo%2F9ss%2B30DqCCOQ0zBAIwA%3D%3D&ppt=sp&ppn=sp&ssid=x9m35hwg280000001680849111872&qH=013c322b6dbb4655"
pm = pro_name(amaz_url)
pm=repl(pm)
filename= "Reviews_"+pm
filename = "/content/drive/MyDrive/web_mining/Reviews/{}.csv".format(filename)
if not os.path.exists(filename):
  collect_data(amaz_url,pm)
print(pm)


Logitech_B170_Wireless_Mouse,_2.4_GHz_with_USB_Nano_Receiver,_Optical_Tracking,_12_Months_Battery_Life,_Ambidextrous,_PC_Mac_Laptop___Black


In [ ]:
i=0
while(True):
  price = check_price(amaz_url)
  price1 = flip_check_price(flip_url)
  if (price <= price1):
    if (price) < 70000:
      email_price(amaz_url,price,"amazon")
      print(price)
      print("The mail is sent (amazon)")
      write_to_csv(pm, price,"amazon")
    else:
      print("the price is not decreased")
      write_to_csv(pm, price,"amazon")
    write_to_csv(pm, price1,"flipkart")
  else:
    if (price1) < 70000:
      email_price(flip_url,price1,"flipkart")
      print(price1)
      print("The mail is sent (flipkart)")
      write_to_csv(pm, price1,"flipkart")
    else:
      print("the price is not decreased")
      write_to_csv(pm, price1,"flipkart")
    write_to_csv(pm, price,"amazon")
  time.sleep((60*60*4))

595
The mail is sent (amazon)


KeyboardInterrupt: ignored